Código para estimar a porcentaxe de apertura dunha Xanela en unha granxa porcina. Nesta implementación empréganse varias técnicas de preprocesado como erosión, dilatación e filtrado Gaussiano; para estimar a porcentaxe aplicanse detectores de bordes de canny e de hough, que permiten ter unha idea de ónde se atopa o listón en cada momento.

In [1]:
###############################
##Autor: Juan López Fernández##
##Data: outubro de 2023      ##
##Materia: Visión Artificial ##
###############################


import cv2
import time
import numpy as np 
from matplotlib import pyplot as plt
import os

In [2]:
#definimos as coordenadas dos puntos nos que se fai click puntosOrixe ,e dos puntos destino da homografía
puntosOrixe = []
puntosDestino = [[0, 0], [500, 0], [500, 255], [0, 255]]

def onClick(event, x, y, flags, param):
    global puntosOrixe

    #gardamos as coordenadas do punto onde fai click o usuario
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(puntosOrixe) < 4:
            puntosOrixe.append([x, y])
            cv2.circle(canvas, (x, y), 3, (0, 255, 0), 2)
            cv2.imshow("primeiro frame", canvas)  #actualiza a ventá para mostrar onde se fixo click

In [3]:
#cargamos o video e capturamos o primeiro frame
source = "../DATA/proba.mp4"
cap = cv2.VideoCapture(source)

#mostramos o primeiro frame e capturamos os eventos do rato sobre o mesmo, 
#pechase a ventá de opencv cando se pulsa algunha tecla
cv2.namedWindow("primeiro frame")
cv2.setMouseCallback("primeiro frame", onClick)

ret, primeiro_frame = cap.read()
canvas = primeiro_frame.copy()
cv2.imshow("primeiro frame", canvas)

cv2.waitKey(0)
if len(puntosOrixe)<4:
    print("Non se capturaron suficientes puntos")
    os.exit()
cv2.destroyAllWindows()

puntosOrixe = np.array(puntosOrixe, dtype=np.int32)
puntosDestino = np.array(puntosDestino, dtype=np.int32)

In [4]:
#buscamos a homografía entre os puntos do usuario e os puntos finais
h, status = cv2.findHomography(puntosOrixe, puntosDestino)

#definimos un kernel que despois empregamos para facer erosión e dilatación
Kernel = (5, 5)
k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, Kernel)

#kernel para axudarnos a eliminar bordes non horizontais detectados por canny
filtroHorizontal=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

#Ventá na que se reproducirá o vídeo
cv2.namedWindow("xanela granxa")

In [5]:
while True:
    ret, frame = cap.read()

    if not ret:
        print("Non hai frame")
        break

    #aplicamos a homografía o frame
    imaxeDestino = cv2.warpPerspective(frame, h, (500, 255))
    l, a, b = cv2.split(cv2.cvtColor(imaxeDestino, cv2.COLOR_BGR2Lab))
    mediaLuz=np.mean(l)
    #convertindo o frame a espazo Lab, podemos saber se é de día ou de noite calculando a media de luz no frame

    #Se hai moita luz, é de día, aplicamos erosión para resaltar o borde da ventá (negro) e atenuar a luz diurna
    if mediaLuz > 100:
        limiar = 50
        frameCorrixido = cv2.erode(imaxeDestino, k1)

    #se é de noite facemos o contrario, resaltamos o borde da ventá (branco porque lle está apuntando unha luz)
    else:
        limiar = 18
        frameCorrixido = cv2.dilate(imaxeDestino, k1)
        
    ##Ademais da erosión ou dilatación, defínese o limiar empregado para o detector de canny

    #aplicamos unha gaussiana para eliminar o ruido aleatorio que hay no video, 
    #sobre todo pola noite onde o efecto é máis notable
    frameCorrixido = cv2.GaussianBlur(frameCorrixido, (9, 9), 2)

    #cun detector de bordes de Canny quedámonos cos bordes do listón da ventá
    bordeXanela = cv2.Canny(frameCorrixido, threshold1=limiar,threshold2=limiar * 3)
    bordeXanela=cv2.filter2D(bordeXanela,-1,filtroHorizontal)

    #aplicamos a transformada de hough os bordes detectados por canny
    #axustando ben o limiar podemos quedarnos só cos bordes horizontais, é dicir, cos do listón
    imaxeDestino=cv2.merge((l,a,b))
    imaxeDestino=cv2.cvtColor(imaxeDestino,cv2.COLOR_Lab2BGR)
    lineas = cv2.HoughLines(bordeXanela, 1, np.pi/180, threshold=160)

    #nesta lista gardanse as distancias de cada liña detectada por hough o borde inferior da xanela de opencv
    distancias=[]

    if lineas is not None:
        for rho, theta in lineas[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))

            cv2.line(imaxeDestino,(x1,y1),(x2,y2),(0,0,255),2)

            distancia=255-y0
            distancias.append(distancia)

    if len(distancias)>0:
        #para saber cómo de pechada está a ventá calculamos a media das distancias o borde inferior da xanela
        #e calculamos a porcentaxe
        meanD=np.mean(distancias)
        aberta=(meanD/puntosDestino[2][1])*100
        cerrada=100-aberta
        
        #hai veces que detecta máis de unha liña inda que só se vexa un borde (cando está pechada)
        #cando o porcentaxe é maior a 92.5 e se ven poucas liñas, significa que está pechada
        if cerrada>92.5 and len(lineas)<=2:
            aberta=0

        cerrada=100-aberta

        cerrada=round(cerrada,2)
        aberta=round(aberta,2)

    #mostramos a porcentaxe de aberta e pechada da ventá da granxa no frame
    imaxeDestino = cv2.putText(imaxeDestino, "pechada: ", (365, 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 0, 128),2, cv2.LINE_AA, False)
    imaxeDestino = cv2.putText(imaxeDestino, "aberta: ", (365, 50),cv2.FONT_HERSHEY_SIMPLEX , 0.5, (128, 0, 128),2, cv2.LINE_AA, False)
    imaxeDestino = cv2.putText(imaxeDestino, str(cerrada)+'%', (445, 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 0, 128),2, cv2.LINE_AA, False)
    imaxeDestino = cv2.putText(imaxeDestino, str(aberta)+'%', (445, 50),cv2.FONT_HERSHEY_SIMPLEX , 0.5, (128, 0, 128),2, cv2.LINE_AA, False)

    cv2.imshow("xanela granxa", imaxeDestino)

    if cv2.waitKey(1000 // 60) == 113:
        print("rematando sesión")
        break

rematando sesión


In [6]:
cap.release()
cv2.destroyAllWindows()